In [ ]:
# import datetime
# import os.path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError




In [ ]:

# # If modifying these scopes, delete the file token.json.
# SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]


# def main():
#     """Shows basic usage of the Google Calendar API.
#     Prints the start and name of the next 10 events on the user's calendar.
#     """
#     creds = None
#     # The file token.json stores the user's access and refresh tokens, and is
#     # created automatically when the authorization flow completes for the first
#     # time.
#     if os.path.exists("token.json"):
#         creds = Credentials.from_authorized_user_file("token.json", SCOPES)
#     # If there are no (valid) credentials available, let the user log in.
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(
#                 "credentials.json", SCOPES
#             )
#             creds = flow.run_local_server(port=0)
#         # Save the credentials for the next run
#         with open("token.json", "w") as token:
#             token.write(creds.to_json())

#     try:
#         service = build("calendar", "v3", credentials=creds)

#         # Call the Calendar API
#         now = datetime.datetime.now(tz=datetime.timezone.utc).isoformat()
#         print("Getting the upcoming 10 events")
#         events_result = (
#             service.events()
#             .list(
#                 calendarId="primary",
#                 timeMin=now,
#                 maxResults=10,
#                 singleEvents=True,
#                 orderBy="startTime",
#             )
#             .execute()
#         )
#         events = events_result.get("items", [])

#         if not events:
#             print("No upcoming events found.")
#             return

#         # Prints the start and name of the next 10 events
#         for event in events:
#             start = event["start"].get("dateTime", event["start"].get("date"))
#             print(start, event["summary"])

#     except HttpError as error:
#         print(f"An error occurred: {error}")


# if __name__ == "__main__":
#     main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=583376317809-bdg1ed3k23m289jb1u84hbsuo6l70k9i.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53568%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=NDGlpbSOHrFw7eLPc4aGrmzAI0Yv3c&access_type=offline


In [ ]:
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langchain_openai import ChatOpenAI

def get_youtube_video_id(url: str) -> str:
    import re
    patterns = [
        r"youtu\.be/([^\?&]+)",
        r"youtube\.com/watch\?v=([^\?&]+)",
        r"youtube\.com/embed/([^\?&]+)",
        r"youtube\.com/v/([^\?&]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    raise ValueError("Invalid YouTube URL")

def get_youtube_title(video_id: str) -> str:
    oembed_url = f"https://www.youtube.com/oembed?url=https://www.youtube.com/watch?v={video_id}&format=json"
    resp = requests.get(oembed_url)
    if resp.status_code == 200:
        return resp.json().get("title", "")
    return ""

def get_youtube_transcript(video_id: str) -> str:
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([x['text'] for x in transcript])
    except (TranscriptsDisabled, NoTranscriptFound):
        return "Transcript not available for this video."

def summarize_text_with_llm(text: str) -> str:
    """
    Summarize the given text using LangChain's ChatOpenAI LLM.
    """
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    prompt = (
        "Summarize the following YouTube video transcript and provide key insights:\n\n"
        f"{text}\n\nSummary:"
    )
    response = llm.invoke(prompt)
    return response.content.strip()

def youtube_video_insight(url: str) -> dict:
    video_id = get_youtube_video_id(url)
    title = get_youtube_title(video_id)
    transcript = get_youtube_transcript(video_id)
    if transcript and transcript != "Transcript not available for this video.":
        summary = summarize_text_with_llm(transcript)
    else:
        summary = "No transcript available to summarize."
    return {
        "title": title,
        "transcript": transcript,
        "summary": summary
    }

# Example usage:
# result = youtube_video_insight("https://www.youtube.com/watch?v=YOUR_VIDEO_ID", "sk-...")
# print(result["title"])
# print(result["summary"])

In [9]:
result = youtube_video_insight("https://www.youtube.com/watch?v=I1rDs_H2iAo")


In [11]:
result

{'title': 'Learn EVERYTHING About FastAPI in Just ONE Project (No Fluff!)',
 'transcript': "if you're tired of watching countless fast API videos and still struggling you came to the right place in this video we are building an intermediate backend social media app using fast API this intermediate project will answer a lot of your fast API questions like how do you connect to a database how do you join tables and configure database tables how do you implement authentication how do you structure a project with routes and how do real life fast API projects even look now this is strictly a guide on how to build an intermediate project from scratch if you want an a toz reference guide on how to use fast API then check out my bestselling fast API course linked below if you're new to the channel I'm Eric Roby a software engineer with over a decade of experience and I've helped thousands of developers learn and grow within their craft now before we dive into code please hit that subscribe but